In [3]:
%pip install -U yolov5

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [4]:
%pip install python-opencv

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement python-opencv (from versions: none)
ERROR: No matching distribution found for python-opencv


In [5]:
import yolov5
import cv2
import numpy
from matplotlib import pyplot as plt


C:\Users\aabid\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# load model
model = yolov5.load('keremberke/yolov5n-construction-safety')
  
# set model parameters
model.conf = 0.25  # NMS confidence threshold
model.iou = 0.45  # NMS IoU threshold
model.agnostic = False  # NMS class-agnostic
model.multi_label = False  # NMS multiple labels per box
model.max_det = 1000  # maximum number of detections per image

YOLOv5  2024-3-27 Python-3.11.5 torch-2.1.1+cu118 CPU

Fusing layers... 
Model summary: 157 layers, 1782166 parameters, 0 gradients, 4.2 GFLOPs
Adding AutoShape... 


In [7]:
def unwrap_detection(input_image, output_data):
    class_ids = []
    confidences = []
    boxes = []

    rows = output_data.shape[0]

    image_width, image_height, _ = input_image.shape

    x_factor = 1024 / 640
    y_factor =  1024 / 640

    for r in range(rows):
        row = output_data[r]
        confidence = row[4]
        if confidence >= 0.4:

            classes_scores = row[5:]
            _, _, _, max_indx = cv2.minMaxLoc(classes_scores)
            class_id = max_indx[1]
            if (classes_scores[class_id] > .25):

                confidences.append(confidence)

                class_ids.append(class_id)

                x, y, w, h = row[0].item(), row[1].item(), row[2].item(), row[3].item() 
                left = int((x - 0.5 * w) * x_factor)
                top = int((y - 0.5 * h) * y_factor)
                width = int(w * x_factor)
                height = int(h * y_factor)
                box = numpy.array([left, top, width, height])
                boxes.append(box)

    return class_ids, confidences, boxes

In [56]:

def video(img):
    for image in img:
        cv2.imshow("detection",image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()
           

In [65]:

def frame(path):

   # Path to video file 
    vidObj = cv2.VideoCapture(path) 

    success , image = vidObj.read()
    
    # count the number of frames
    count = 0

    # checks whether frames were extracted 
    success = 1
    print(vidObj.get(cv2.CAP_PROP_FPS))
    
    img =[]

    # while succes:
    while vidObj.isOpened():

        success , image = vidObj.read()
        size =  image.shape

        if success :
       
          # perform inference
          results = model(image, size=240)

          # inference with test time augmentation
          # results = model(image, augment=True)

          # parse results
          predictions = results.pred[0]
          boxes = predictions[:, :4] # x1, y1, x2, y2
          scores = predictions[:, 4]
          categories = predictions[:, 5]
        


          # show detection bounding boxes on image
          # results.show()

          img.append(numpy.squeeze(results.render()))
          cv2.imshow("detection",numpy.squeeze(results.render()))
          
          if cv2.waitKey(1) & 0xFF == ord('q'):
                break

          
          # plt.imshow(numpy.squeeze(results.render()))
          # plt.show()
          # print(results.render())

          count +=1
    vidObj.release()
    cv2.destroyAllWindows()
    
   
    

In [67]:
frame('./video/Rummel.mp4')

23.976023976023978


In [11]:
import os

def convert_avi_to_mp4(avi_file_path, output_name):
    os.popen("ffmpeg -i '{input}' -ac 2 -b:v 2000k -c:a aac -c:v libx264 -b:a 160k -vprofile high -bf 0 -strict experimental -f mp4 '{output}.mp4'".format(input = avi_file_path, output = output_name))
    return True


In [12]:
convert_avi_to_mp4('.\video3.avi',output_name='output')

True